In [5]:
import random
import numpy as np

In [6]:
dataset=[10,11]
learningRate=0.0002
errorThresh=1

class Layer:
    
    def __init__(self,neuronCount=None,position=None,neuronvals=None):
        
        self.neuronCount = neuronCount
        self.position = position
        self.neuronvals = []
        self.weights = []
        self.bias = []
        self.NodeDeltas=[] 

        #set by setintialweights and setnodedelta
        self.previousLayer =None
        self.AfterLayer =None

        for i in range(self.neuronCount):

            # self.bias.append(random.uniform(0.0001,1))
            self.bias.append(1)

        if position==1:
            self.neuronvals = neuronvals
    
    def setInitialWeights(self,previousLayer):

        self.previousLayer = previousLayer
        test =[
            [1,2,3,4]
        ]
        for i in range(self.neuronCount):
            temp = []
            for j in range(previousLayer.neuronCount):
                temp.append(random.uniform(0.0001,1))
                # temp.append(test[0][i+j])

            self.weights.append(temp)

    def setNodeDelta(self,AfterLayer):
        """""
        set nodedelta value for layers
        """""
        self.AfterLayer = AfterLayer
        if self.position == -1:
            # outputlayer
            if len(self.NodeDeltas)==0:
                for i in range(self.neuronCount):

                    nodedelta = self.neuronvals[i]-dataset[i] #exepectd to change dimenston CHNGDIMN
                    self.NodeDeltas.append(nodedelta)
            else:
                for i in range(self.neuronCount):

                    nodedelta = self.neuronvals[i]-dataset[i] #exepectd to change dimenston CHNGDIMN
                    self.NodeDeltas[i]=nodedelta
        else:
            # hidden layers
            if len(self.NodeDeltas)==0:

                for i in range(self.neuronCount):
                    nodedelta= 0
                    for j in range(AfterLayer.neuronCount):
                        nodedelta = nodedelta+AfterLayer.NodeDeltas[j]*self.weights[i][j]
                    self.NodeDeltas.append(nodedelta)

            else:
                    for i in range(self.neuronCount):
                        nodedelta= 0
                        for j in range(AfterLayer.neuronCount):
                            nodedelta = nodedelta+AfterLayer.NodeDeltas[j]*self.weights[i][j]
                        self.NodeDeltas[i]=nodedelta
    
    def updateWeightsandBias(self):

        len_weight = len(self.weights[0])# based on previous layer of no(neurons) lenof weights is set

        for i in range(self.neuronCount):

            for j in range(len_weight):
                
                new_Weight = self.weights[i][j] - learningRate * self.NodeDeltas[i] * self.previousLayer.neuronvals[j]
                self.weights[i][j] = new_Weight
            
            new_Bias = self.bias[i] - learningRate * self.NodeDeltas[i]
            self.bias[i]=new_Bias        

In [7]:
class Ann:

    def __init__(self):
        self.LayerArr = []
        self.batchSize = -1

    def strap(self,layer1=None,layer2=None):
        "sets the initial weights of the ANN"
        layer2.setInitialWeights(layer1)

    def forward(self,layer1=None,layer2=None):
        "performs forward passing for all layers in network"  
        if len(layer2.neuronvals)==0:
            for i in range(layer2.neuronCount):
                layer2.neuronvals.append(np.dot(layer1.neuronvals,layer2.weights[i])+layer2.bias[i])
        else:
            for i in range(layer2.neuronCount):
                layer2.neuronvals[i]=np.dot(layer1.neuronvals,layer2.weights[i]+layer2.bias[i])
            
    def backPropagate(self,currentLayer=None,AfterLayer=None):
        """""
        performs back propagation by

        1. seting nodeDelta (a function in lasyer class)
        2.Update werights and biases (a function in layer class)
        
        """""
        
        
        
        currentLayer.setNodeDelta(AfterLayer)
        currentLayer.updateWeightsandBias()
        
    def compile(self,batchSize=-1):
        """""
        calls strap function for every layer : initilizes weights in Ann
        """""
        self.batchSize = batchSize
        for i in range(len(self.LayerArr)-1):
            self.strap(layer1=self.LayerArr[i],layer2=self.LayerArr[i+1])
    
    def findError(self):
        error = 0
        for i in range(len(self.LayerArr[-1].neuronvals)):#CHNGDIMN
            error = error+(dataset[i]-self.LayerArr[-1].neuronvals[i])**2
        error = 0.5*error
        return error
    
    def Train(self):
        """
        Trains the model 
        
        """
        errorrate = 9999999 
        layercount = len(self.LayerArr)

        while (errorrate > errorThresh):

            #forward propagation
            for i in range(layercount-1):
                self.forward(layer1=self.LayerArr[i],layer2=self.LayerArr[i+1])
            
            #error analysis
            errorrate = self.findError()
            # print(errorrate)
            if(errorrate <errorThresh):
                print("optimised")
                print(self.LayerArr[-1].neuronvals)
                break
          

            #backward propagation
            currentLayeridx = layercount-1  
            for i in range(layercount-1):

                if self.LayerArr[currentLayeridx].position ==-1:# handling output layer 
                    self.backPropagate(currentLayer=self.LayerArr[currentLayeridx])
                    print(self.LayerArr[currentLayeridx].neuronvals)
                    currentLayeridx=currentLayeridx-1
                else:
                    self.backPropagate(currentLayer=self.LayerArr[currentLayeridx],AfterLayer=self.LayerArr[currentLayeridx+1])
                    currentLayeridx=currentLayeridx-1
                              
    def bind(self,layer=None):
        """
        use this function to add layers into network
        
        if input layer give coresponding value too
        """
        self.LayerArr.append(layer)

In [8]:

# inputl = Layer(2,1,[2,3])
inputl = Layer(neuronCount=2,position=1,neuronvals=[2,3])
hidden = Layer(neuronCount=2,position=2,neuronvals=[])
# hidden2 = Layer(neuronCount=2,position=3,neuronvals=[]) 
output = Layer(neuronCount=2,position=-1,neuronvals=[])

Ann = Ann()
Ann.bind(layer=inputl)
Ann.bind(layer=hidden)
# Ann.bind(layer=hidden2)
Ann.bind(layer=output)
Ann.compile()
Ann.Train()

[3.0220934142793494, 3.6935470208219914]
[18.507526165254802, 19.720686320850596]
[18.26604774476737, 19.46907178774636]
[18.033565602234912, 19.226918468928293]
[17.809617999659093, 18.99373907460006]
[17.593774780828408, 18.76907967394343]
[17.38563466789653, 18.55251683889421]
[17.184822833699375, 18.343655079273994]
[16.990988717365077, 18.142124534991915]
[16.803804055078494, 17.947578895582513]
[16.622961101534734, 17.759693521227092]
[16.44817102075686, 17.57816374272489]
[16.279162427645836, 17.4027033207261]
[16.115680063946108, 17.23304304698549]
[15.957483594305852, 17.06892947250404]
[15.804346509835202, 16.910123749248488]
[15.656055128059597, 16.756400573716718]
[15.512407679461859, 16.607547221987367]
[15.3732134719351, 16.46336266708427]
[15.238292125452391, 16.323656770525922]
[15.107472870118892, 16.18824954083896]
[14.9805939015251, 16.056970452609807]
[14.857501787980405, 15.929657820347117]
[14.73805092478679, 15.806158222040786]
[14.622103031224064, 15.68632596784